In [1]:
import numpy as np
import os

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import mpl_toolkits
#from mpl_toolkits.mpot3d import Axes3D
import time



import tensorflow as  tf

In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

Linear model 


$y(x) = a + b_1 * X_1 + b_2*X_2 + b_3*X_3 + \sigma \epsilon$

$\epsilon \approx N(0,1)$

$b_1, b_2, b_3 = (0.5, 0.2, 0.1) $

$\sigma=0.1$


$X_1, X_2, X_3$ uniformally distributed in [-1,1]

In [3]:
#Generate Data

n_points = 5000
n_features = 3

bias = np.ones(n_points).reshape((-1,1))
low  = -1 * np.ones((n_points, n_features),dtype='float')
high  = np.ones((n_points, n_features),dtype='float')

#similated Features
X =  np.random.uniform(low=low, high=high)


#simulated Noise
noise = np.random.normal(size=(n_points,1))

#outputr
weights = np.array([1.0, 0.5, 0.2, 0.1])
noise_std = 0.1

Y = weights[0] * bias + np.dot(X, weights[1:] .reshape(-1,1)) + noise_std * noise


In [5]:
#View X
print(X.shape)
X

(5000, 3)


array([[ 0.15781829,  0.13063534,  0.74025826],
       [ 0.28604545, -0.97095358, -0.57869739],
       [ 0.47870732,  0.07846252,  0.84635806],
       ..., 
       [ 0.94591511, -0.86843918,  0.13608197],
       [ 0.41194819, -0.01217022, -0.6760082 ],
       [ 0.31758177,  0.87903769,  0.37626833]])

In [6]:
#View Y
print(Y.shape)
Y

(5000, 1)


array([[ 1.10968022],
       [ 0.89234434],
       [ 1.34779366],
       ..., 
       [ 1.24371334],
       [ 1.13320279],
       [ 1.33009926]])

In [7]:
#split the data into train and test sets
train_test_split = 4

n_test = int(n_points/train_test_split)
n_train = n_points - n_test

X_train = X[:n_train, :]
Y_train = Y[:n_train].reshape((-1,1))

X_test = X[n_train:,:]
Y_test = Y[n_train:].reshape((-1,1))

In [8]:
X_test.shape, Y_test.shape, X_train.shape, Y_train.shape

((1250, 3), (1250, 1), (3750, 3), (3750, 1))

In [9]:
#tf.placeholder(dtype, shape=None, name=None)

In [38]:
#We can do directly - but let us use Minibatch GRadient Descent
#tf.placeholder(dtype, shape=None, name=None)

reset_graph()

n_epochs = 500
learning_rate = 0.01


X = tf.placeholder(tf.float32, shape=(None, n_features + 1), name = "X")
y = tf.placeholder(tf.float32, shape=(None, 1) , name = "y")
theta = tf.Variable(tf.random_uniform([n_features + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y

mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = 2.0/n_train * tf.matmul(tf.transpose(X), error)

training_op = tf.assign(theta, theta - learning_rate* gradients)

#Then define the batch size and compute the number of batches
#batch_size = 100
#n_batches  = np.ceil(n_test, batch_size)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: np.hstack([np.ones(X_train.shape[0]).reshape(-1,1),X_train]), 
                                         y: Y_train})
        
    best_theta = theta.eval()
    
    
    

In [39]:
best_theta

array([[ 1.00069797],
       [ 0.47194025],
       [ 0.17396879],
       [ 0.11091298]], dtype=float32)

In [40]:
#Diff from actual 
np.round(weights.reshape(-1,1) - best_theta,3)

array([[-0.001],
       [ 0.028],
       [ 0.026],
       [-0.011]])

In [46]:
logdir

'tf_logs/run-20180805063540/'

In [52]:
#Visualizing Using the tensorboard

from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
#'tf_logs/run-20180805063540/'


reset_graph()

n_epochs = 500
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n_features + 1), name = "X")
y = tf.placeholder(tf.float32, shape=(None, 1) , name = "y")
theta = tf.Variable(tf.random_uniform([n_features + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y

mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = 2.0/n_train * tf.matmul(tf.transpose(X), error)
  
training_op = tf.assign(theta, theta - learning_rate* gradients)

#Then define the batch size and compute the number of batches
#batch_size = 100
#n_batches  = np.ceil(n_test, batch_size)

mse_summary = tf.summary.scalar("MSE", mse)
file_witer  = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: np.hstack([np.ones(X_train.shape[0]).reshape(-1,1),X_train]), 
                                         y: Y_train})
        if(epoch %10 == 0):
            summary_str = mse_summary.eval(feed_dict={X: np.hstack([np.ones(X_train.shape[0]).reshape(-1,1),X_train]), 
                                         y: Y_train})
            file_witer.add_summary(summary_str)
        
    best_theta = theta.eval()
    
    
    

In [51]:
n_epochs

500